In [ ]:
import torch
import numpy as np

import os, sys
project_root_dir = os.path.join(os.getcwd(),'../..')
if project_root_dir not in sys.path:
    sys.path.append(project_root_dir)
    import config

In [ ]:
from FeatureSelection.Experiments.doc import ExperimentDoc

def generate_figures(exp) -> None:
    save_dir = os.path.join(exp.config()['save_result_dir'], 'imgs')
    
    # Histrogram
    hist_fig = ExperimentDoc.hist_phi(exp)
    hist_fig.savefig(os.path.join(save_dir, 'hist_phi.pdf'), bbox_inches='tight')

    # Band Selected
    dataset = exp.dataset
    X, y = dataset[:]

    # get 1 samper per class
    samples = np.zeros((3, 128))
    sample_labels = ['Healthy', 'Tumor', 'Hypervascular']
    for i in range(0, 3):
        idx = np.where(y == i)[0][0]
        samples[i] = X[idx].numpy()

    samples = dataset.transform.inverse_transform(torch.tensor(samples))
    wv = np.round(dataset.wv, 0).astype(int)

    selected_band_fig = ExperimentDoc.plot_band_selection(exp, samples=samples, labels=sample_labels, wv=wv)
    selected_band_fig.savefig(os.path.join(save_dir, 'selected_band.pdf'), bbox_inches='tight')


In [ ]:
from FeatureSelection.Experiments.BrainCancerHSIBandSelection import VNIRimagesOp8C1
experiments = [VNIRimagesOp8C1]

for _exp in experiments:
    exp = _exp()
    exp.run()
    exp.save_config()
    exp.save_results()
    generate_figures(exp)

In [ ]:
from FeatureSelection.Experiments.BrainCancerHSIBandSelection import VNIRimagesOp12C1
experiments = [VNIRimagesOp12C1]

for _exp in experiments:
    exp = _exp()
    exp.run()
    exp.save_config()
    exp.save_results()
    generate_figures(exp)

In [ ]:
from FeatureSelection.Experiments.BrainCancerHSIBandSelection import VNIRimagesOp15C1, VNIRimagesOp20C1, VNIRimagesOp12C2, VNIRimagesOp8C2
experiments = [VNIRimagesOp15C1, VNIRimagesOp20C1, VNIRimagesOp12C2, VNIRimagesOp8C2]

for _exp in experiments:
    exp = _exp()
    print(exp.experiment)
    exp.run()
    exp.save_config()
    exp.save_results()
    generate_figures(exp)

In [ ]:
from FeatureSelection.Experiments.BrainCancerHSIBandSelection import VNIRimagesOpAll_KMeans, VNIRimagesOpAll_OCSP
experiments = [VNIRimagesOpAll_KMeans, VNIRimagesOpAll_OCSP]

for _exp in experiments:
    exp = _exp()
    print(exp.experiment)
    exp.run()
    exp.save_config()
    exp.save_results()
    generate_figures(exp)